### Generate_san_vw_with_rules
For train,test,dev files

In [6]:
# import fasttext
# model = fasttext.load_model("../../fastText/result/cc.sanskrit.50.bin")

In [2]:
import pandas as pd
# data = pd.read_csv('../../DCST/data/Rules/rule_'+'Gold_1496',header=None,sep=',')
# data

In [3]:
# load = pd.read_csv('../../DCST/data/sanskrit_treebank/'+'dev/'+'Gold_1496'+'.csv',header=None,sep=',')
# load

In [5]:
import re
indeclinable = ['ind','prep','interj','prep','conj','part','abs','ca abs']
case_list = ['nom','voc','acc','i','dat','abl','g','loc']
gender_list = ['n','f','m','*']
person_list = ['1','2','3']
no_list = ['du','sg','pl']
pops = [' ac',' ps']
def process_ma(ma):
          # Variation:III
        temp = re.sub("([\(\[]).*?([\)\]])", "\g<1>\g<2>", ma).replace('[] ','').strip(' []')
        temp = temp.split('.')
        if temp[-1] == '':
            temp.pop(-1)
        # Remove active passive
        case=''
        no=''
        person=''
        gender=''
        tense=''
        
        for a,b in enumerate(temp):
            if b in pops:
                temp.pop(a)
        # Get gender
        for a,b in enumerate(temp):
            if b.strip() in gender_list:
                gender = b.strip()
                temp.pop(a)
        # Get case
        for a,b in enumerate(temp):
            if b.strip() in case_list:
                case = b.strip()
                temp.pop(a)
        # Get person
        for a,b in enumerate(temp):
            if b.strip() in person_list:
                person = b.strip()
                temp.pop(a)
        # Get no
        for a,b in enumerate(temp):
            if b.strip() in no_list:
                no = b.strip()
                temp.pop(a)
        # Get Tense
        for b in temp:
            tense=tense+ ' '+b.strip()
        tense=tense.strip()
        return case, no, gender, tense, person
print(process_ma('prep. pr. [1] ac. du. 3'))

('', 'du', '', 'prep pr', '3')


In [17]:
def get_string(case, no, gender, tense, person):
    var=''
    if case:
        t = 'case=%s '%case
        var=var+t
    if gender:
        t = 'gender=%s '%gender
        var=var+t
    if tense:
        t = 'tense=%s '%tense
        var=var+t
    if no:
        t = 'no=%s '%no
        var=var+t
    if person:
        t = 'case=%s '%person
        var=var+t
#         print(var)
    return var

In [18]:
def get_uoh_string(store):
    store_case, store_no, store_gender, store_tense, store_person = '','','','',''
    for ma in store:
        case, no, gender, tense,person = process_ma(ma)
        store_case+= ' '+case
        store_no+= ' '+no
        store_gender+= ' '+gender
        store_tense+= ' '+tense
        store_person+= ' '+person
    var = get_string(store_case.strip(), store_no.strip(), store_gender.strip(),\
                     store_tense.strip(), store_person.strip())
    return var

In [20]:
inpu = "pft. md. pl. 3@pft. md. pl. 3@nom. sg. n.@nom. pl. n."
inpu = inpu.lstrip('@')
inpu= inpu.split('@')
print(get_uoh_string(inpu))

case=nom nom gender=n n tense=pft md pft md no=pl pl sg pl case=3 3 


In [10]:
def get_rule(load,file, line_no):
#     print('function: ',file,line_no)
    head=''
    pos=''
    tag=''
    try:
        df = pd.read_csv('../../DCST/data/Rules/rule_'+file,header=None,sep=',')
    except:
        return head,pos,tag
    for i in range(len(df)):
        if df.iloc[i,1] == int(line_no):
            head+= ' '+str(load.iloc[df.iloc[i,4],1])
            pos+= '@'+str(load.iloc[df.iloc[i,4],3])
            tag+= ' '+str(df.iloc[i,8])
    return head,pos,tag

    
# print(get_rule(load,'Gold_1496',2))

In [22]:
import pandas as pd
# Get all the tags in hash list
hash = {}
tag_file_name = '../data/san_tags'
# input_file_name = './sanskrit_treebank/ud_pos_ner_dp_train_san'
files = ['ud_pos_ner_dp_dev_san','ud_pos_ner_dp_train_san','ud_pos_ner_dp_test_san']

def readtags():
    for line in open(tag_file_name).readlines():
        hash[line.split()[0]] = int(line.strip().split()[1])
c = 1
readtags()
for input_file_name in files:
    if 'train' in input_file_name:
        target = 'train/'
    elif 'test' in input_file_name:
        target = 'test/'
    else:
        target = 'dev/'
    data = open('../data/'+input_file_name).readlines()
    writer = open('../models/Dep_san_temp/'+input_file_name+'_with_rules'+'.vw','w')
    for line in data:
#         print('Line',line.replace('\n',''))
        if line == '\n':
            writer.write('\n')
            continue
        splits = line.strip().split('\t')

        # This replace can be removed.
        strw = "|w %s"%splits[1].replace(":","COL");
        strp = "|p %s"%splits[2].replace(":","COL");
#         
#         print(head,tag)
        ##############################################################
        # To pass gold data use this blog
        if 'test' in input_file_name:
            load = pd.read_csv('../../DCST/data/sanskrit_treebank/'+'test/'+splits[3]+'.csv',header=None,sep=',')
        else:
            try:
                load = pd.read_csv('../../DCST/data/sanskrit_treebank/'+'train/'+splits[3]+'.csv',header=None,sep=',')
            except:
                load = pd.read_csv('../../DCST/data/sanskrit_treebank/'+'dev/'+splits[3]+'.csv',header=None,sep=',')
#         print(load.iloc[int(splits[4])-1,1],load.iloc[int(splits[4])-1,3])
#         if int(splits[4]) != 0:
#             hrule= "|hrule %s"%str(load.iloc[int(splits[4]),1])
#             prule= "|prule %s"%str(load.iloc[int(splits[4]),3])
#         else:
#             hrule= "|hrule %s"%splits[1]
#             prule= "|prule %s"%splits[2]
#         ttule= "|trule %s"%splits[-1]
        ###############################################################
        # To pass samsadhini output use this function
        # Variation I
#         head,pos,tag =get_rule(load,splits[3],splits[0])
#         h = "|hrule %s"%head;
#         p = "|prule %s"%pos;
#         t = "|trule %s"%tag;
        ###############################################################
#         # Variation II
#         head,pos,tag =get_rule(load,splits[3],splits[0])
#         h = " hrule=%s"%head.strip();
#         p = " prule=%s"%pos.strip();
#         t = " trule=%s"%tag.strip();
#         var ="|uoh"
#         if head:
#             var = var+h
#         if pos:
#             var =var+p
#         if tag:
#             var =var+t
        #############################################################
#         Variation III
        head,pos,tag =get_rule(load,splits[3],splits[0])
        case, no, gender, tense, person = process_ma(splits[2])
        var =get_string(case, no, gender, tense, person)
        strp = "|pos %s"%var.strip();
        h = "|hrule %s"%head.strip();
        t = "|trule %s"%tag;
        pos = pos.lstrip('@')
        pos= pos.split('@')
        p = "|prule %s"%get_uoh_string(pos);

        
#         print(hrule)
#         word = model.get_word_vector(splits[1]);
        ################################################################
        # Add fastText
#         feature = ''
#         i=0
#         for v in word:
#             feature = feature+' '+str(v)
# #             feature = feature+' '+'F'+str(i)+':'+str(v)

#             i=i+1
#         embedding = "|FastText%s"%feature;
#         strq = "|FASTTEXT T0:%s U1:%s"%(word[1],word[1]);
        tag = splits[-1]
        if tag not in hash:
            hash[tag] = c
            c+=1

        #writer.write('%s 1.0  %s:%s%s %s\n'%((int(splits[7])+1) + (hash[tag]<<8), int(splits[7]),tag,strw, strp))
        # Writing in order
        # head, tag_integer mapping, head, tag, word, pos
        writer.write('%s %s %s:%s%s %s %s %s %s\n' % (int(splits[4]), hash[tag], int(splits[4]), tag, strw, strp, h,p,t))
    writer.close()
print('Value of out of voc tag ',c)
print('successful')

Value of out of voc tag  1
successful
